## Purpose

Characterize the sequences in Nef RM9 epitope. Processes a folder hierarchy of FASTQ files and saves Nef RM9 sequences, counts and statistics files to a corresponding folder hierarchy. This script will generate a tsv file with nucleotide sequences and the number of times they are present in the data set. 

## Specify dataset arguments

Specify variables that need to be changed depending on the dataset and machine configuration. After these parameters are specified, the rest of the notebook should be runnable automatically.

In [1]:
# path to FASTQ files to process
# expects R1 and R2 paired samples
FASTQ_FOLDER_PATH = 'Used_Files/'

# path to folder containing barcode counts, intermediate files, and statistics
OUTPUT_PATH = 'RM9_counts/'

# path to bbmap
BBMAP_PATH = 'bin/bbmap/'

# path to reference file for read orientation
REF_FASTA = 'ref/M33262.fasta'

# 20bp of upstream and downstream sequences flanking Nef RM9
UPSTREAM_FLANKING = 'ACTTGGTAGGGGTATCAGTG'
DOWNSTREAM_FLANKING = 'AGTTACAAATTGGCAATAGA'

## Functions

Define functions that will be used in the workflow to count barcodes from FASTQ files.

In [2]:
# import logger

def print_status(status):
    '''print timestamped status update'''
    
    from datetime import datetime
    import logging
    
    log = logging.getLogger(__name__)
    
    print('--[' + datetime.now().strftime('%Y-%m-%d %H:%M:%S') + '] ' + status + '--')
    log.info(status)

def create_temp_file():
    '''create named temporary file
    return temporary file object [0] and path to temporary file [1]'''

    import pathlib
    import tempfile
    
    temp = tempfile.NamedTemporaryFile()    
    return [temp, temp.name]

def create_directory(dir_path):
    '''create directory at specified location if one does not already exist'''
    
    import os
    
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
        
    return dir_path

def copy_and_overwrite(from_path, to_path):
    '''duplicate folder and file hierarchy'''
    import os
    import shutil
     
    print_status('Copying files from ' + from_path + ' to ' + to_path)
    if os.path.exists(to_path):
        shutil.rmtree(to_path)
    shutil.copytree(from_path, to_path)
    
def find_files_in_path(search_path):
    '''get list of files matching files in search_path'''
    import glob
    
    f = glob.glob(search_path, recursive=True)
    
    return f

def derive_file_name(source_file, find_string, replace_string):
    '''create filename string '''
    
    f = source_file.replace(find_string, replace_string)
    
    return f

def run_command(cmd_list, stdout_file = None, stderr_file = None):
    '''run command with subprocess.call
    if stdout or stderr arguments are passed, save to specified file
    '''
    
    import subprocess
    
    print_status(' '.join(cmd_list)) # print status
    
    # if neither stdout or stderr specified
    if stdout_file is None and stderr_file is None:
        print(cmd_list)
        subprocess.call(cmd_list)
     
    # if only stdout is specified
    elif stdout_file is not None and stderr_file is None:
        with open(stdout_file, 'w') as so:
            subprocess.call(cmd_list, stdout = so)
     
    # if only stderr is specified
    elif stdout_file is None and stderr_file is not None:
        with open(stderr_file, 'w') as se:
            subprocess.call(cmd_list, stderr = se)
     
    # if both stdout and stderr are specified
    elif stdout_file is not None and stderr_file is not None:
        with open(stdout_file, 'w') as so:
            with open(stderr_file, 'w') as se:
                subprocess.call(cmd_list, stdout = so, stderr = se)
    
    else: pass
    
def split_path(source_file):
    '''get path to enclosing folder and basename for source_file'''
    
    import os
    
    source_file_path = os.path.dirname(source_file)
    source_file_basename = os.path.basename(source_file)
    
    return [source_file_path, source_file_basename]

def create_barcode_tsv(two_column_barcode_count_file, three_column_barcode_count_file, sample_name):
    '''add a column with sample_name to barcode count TSV output by kmercountexact'''
    
    import pandas as pd 
    
    # import file containing counts to pandas dataframe    
    df = pd.read_csv(two_column_barcode_count_file, sep='\t', names=['barcode_sequence', 'barcode_count'])
    
    # add column with sample name
    df.insert(0, 'sample_name', sample_name)
    
    # export CSV with sample names added
    df.to_csv(three_column_barcode_count_file, sep='\t', index=False)
    
def remove_file(source_file):
    '''remove file'''
    
    import os
    
    os.remove(source_file)

## Workflow

Count numbers of barcodes in FASTQ sequences by:

1. Create folders to store barcode counts, intermediate files, and tools statistics
1. Make dictionary of samples to process
1. Iterate over every sample to process and:
1. Quality trimming and merging R1 and R2 FASTQ sequences
1. Extracting reads that contain the barcode
1. Orient reads in same direction
1. Trim reads to contain only the barcode
1. Count number of times each barcode occurs
1. Create TSV file with barcode counts for each sequence

### 1.

Checks if the output folder designated by the user exists, and creates it if not. The function then generates the output folder structure based on the input folder structure.

In [3]:
## 1. Create folders to store barcode counts, intermediate files, and tools statistics ##

copy_and_overwrite(from_path = FASTQ_FOLDER_PATH, to_path = OUTPUT_PATH)



--[2019-09-20 09:19:03] Copying files from Used_Files/ to RM9_counts/--


### 2. 

Creates a dictionary of sample names, where the key is the sample directory path and the value is the file name sans forward/reverse designator and file extension.

In [4]:
## 2. Make dictionary of samples to process ##

FASTQ_R1 = find_files_in_path(OUTPUT_PATH + '/**/*_R1_001.fastq.gz')

SAMPLES = {}

for i in FASTQ_R1:
    
    # get path to directory containing sample FASTQ
    SAMPLE_DIR = split_path(i)[0]
    
    # get sample name
    FASTQ_R1_BASENAME = split_path(i)[1]
    SAMPLE_NAME = derive_file_name(FASTQ_R1_BASENAME, '_R1_001.fastq.gz', '')
    
    SAMPLES[i] = (SAMPLE_DIR, SAMPLE_NAME)

### 3. 

Iterates over every sample in the dictionary to create a stats file and...

### 4. 

Merges and quality trims the forward and reverse FASTQ reads using bbmerge. 

### 5. 

- Extracts reads containing the user-defined upstream flanking sequence, allowing for a 1 bp mismatch.
- From those reads, extracts reads containing the user-defined downstream flanking sequence, again allowing for a 1 bp mismatch.

### 6. 

- Maps reads to Zika virus reference using bbmap.
- Using bbmap's splitsam utility, identifies plus and minus strands in the output sam file. Regenerates FASTQ files for plus and minus reads, and takes the reverse complement of the minus strands before concatenating the two files into a single FASTQ.

### 7.

Using bbduk, removes the upstream and downstream flanking sequences, again allowing for a 1 bp mismatch. By orienting the reads in Step 6, the upstream sequence can be left-trimmed and the downstream sequence can be right-trimmed without needing to account for reverse complements.

### 8. 

Counts the number of times each 24mer barcode appears in each sample. By limiting to 24 bp, any PCR chimeras containing > 24 bp between flanking sequences are removed.

### 9.

Outputs a TSV file with barcode counts for each sequence, and removes temporary files created in the above steps. 


In [5]:
## 3. Iterate over every sample to process and: ##

for key,val in SAMPLES.items():
    
    # get sample path and sample name
    SAMPLE_PATH = val[0]
    SAMPLE_NAME = val[1]
    
    # create directory to hold statistics files
    STATS_PATH = create_directory(SAMPLE_PATH + '/stats')

    ## 4. Quality trimming and merging R1 and R2 FASTQ sequences ##
    
    # merge and quality trim
    run_command([BBMAP_PATH + 'bbmerge.sh',
                 'in=' + SAMPLE_PATH + '/' + SAMPLE_NAME + '_R1_001.fastq.gz',
                 'in2=' + SAMPLE_PATH + '/' + SAMPLE_NAME + '_R2_001.fastq.gz',
                 'out=' + SAMPLE_PATH + '/' + SAMPLE_NAME + '.merged.fastq.gz',
                 'qtrim=t',
                 'ow=t'],
                 stderr_file = STATS_PATH + '/' + SAMPLE_NAME + '.merging.stats.txt')
    
    ## 5. Extracting reads that contain the barcode ##
    
    # extract reads containing barcode
    # bbduk command to find reads containing upstream flanking sequence   
    run_command([BBMAP_PATH + 'bbduk.sh',
                  'in=' + SAMPLE_PATH + '/' + SAMPLE_NAME + '.merged.fastq.gz',
                  'outm=' + SAMPLE_PATH + '/' + SAMPLE_NAME + '.containing_upstream_flank.fastq.gz',
                  'literal=' + UPSTREAM_FLANKING,
                  'k=20',
                  'hdist=1',
                  'ow=t'],
                  stderr_file = STATS_PATH + '/' + SAMPLE_NAME + '.containing_upstream_flank.stats.txt')
    
    # bbduk command to find reads containing downstream flanking sequence 
    # (among those that contain upstream flanking sequence)
    run_command([BBMAP_PATH + 'bbduk.sh',
                  'in=' + SAMPLE_PATH + '/' + SAMPLE_NAME + '.containing_upstream_flank.fastq.gz',
                  'outm=' + SAMPLE_PATH + '/' + SAMPLE_NAME + '.containing_barcode.fastq.gz',
                  'literal=' + DOWNSTREAM_FLANKING,
                  'k=20',
                  'hdist=1',
                  'ow=t'],
                  stderr_file=STATS_PATH + '/' + SAMPLE_NAME + '.containing_barcode.stats.txt')
    
    ## 6. Orient reads in same direction ##title
    
    # orient reads by mapping to reference
    # use SIV reference with accession M33262
    run_command([BBMAP_PATH + 'bbmap.sh',
                'in=' + SAMPLE_PATH + '/' + SAMPLE_NAME + '.containing_barcode.fastq.gz',
                'outm=' + SAMPLE_PATH + '/' + SAMPLE_NAME + '.mapped.sam',
                'ref=' + REF_FASTA,
                'ow=t'],
                stderr_file=STATS_PATH + '/' + SAMPLE_NAME + '.mapping.stats.txt')
    
    # split mapped SAM file into plus, minus, and unmapped strand SAM temporary files
    PLUS_STRAND_SAM = create_temp_file()
    MINUS_STRAND_SAM = create_temp_file()
    UNMAPPED_STRAND_SAM = create_temp_file()
    
    run_command([BBMAP_PATH + 'splitsam.sh',
                SAMPLE_PATH + '/' + SAMPLE_NAME + '.mapped.sam',
                PLUS_STRAND_SAM[1] + '.sam',
                MINUS_STRAND_SAM[1] + '.sam',
                UNMAPPED_STRAND_SAM[1] + '.sam'])    
    
    # reformat plus and minus strand SAM files to FASTQ
    PLUS_STRAND_FASTQ = create_temp_file()
    MINUS_STRAND_FASTQ = create_temp_file()    

    run_command([BBMAP_PATH + 'reformat.sh',
                'in=' + PLUS_STRAND_SAM[1] + '.sam',
                'out=' + PLUS_STRAND_FASTQ[1] + '.fastq.gz'])
                 
    run_command([BBMAP_PATH + 'reformat.sh',
                'in=' + MINUS_STRAND_SAM[1] + '.sam',
                'out=' + MINUS_STRAND_FASTQ[1] + '.fastq.gz',
                'rcomp=t'])
    
    # concatenate oriented plus and minus strand FASTQ
    run_command(['cat',
                PLUS_STRAND_FASTQ[1] + '.fastq.gz',
                MINUS_STRAND_FASTQ[1] + '.fastq.gz'],
               stdout_file = SAMPLE_PATH + '/' + SAMPLE_NAME + '.oriented.fastq.gz')
    
    ## 7. Trim reads to contain only the barcode ##
    
    # remove upstream flanking sequence from oriented reads
    UPSTREAM_FLANK_REMOVED = create_temp_file()
    
    run_command([BBMAP_PATH + 'bbduk.sh',
               'in=' + SAMPLE_PATH + '/' + SAMPLE_NAME + '.oriented.fastq.gz',
               'out=' + UPSTREAM_FLANK_REMOVED[1] + '.fastq.gz',
               'literal=' + UPSTREAM_FLANKING,
               'ktrim=l',
               'rcomp=f',
               'k=20',
               'hdist=1',
               'ow=t'],
               stderr_file = STATS_PATH + '/' + SAMPLE_NAME + '.upstream.flank.removal.stats.txt')
    
    # remove downstream flanking sequence from oriented reads
    run_command([BBMAP_PATH + 'bbduk.sh',
               'in=' + UPSTREAM_FLANK_REMOVED[1] + '.fastq.gz',
               'out=' + SAMPLE_PATH + '/' + SAMPLE_NAME + '.barcodes.fastq.gz',
               'literal=' + DOWNSTREAM_FLANKING,
               'ktrim=r',
               'rcomp=f',
               'k=20',
               'hdist=1',
               'minlength=27',  
               'maxlength=27',
               'ow=t'],
               stderr_file = STATS_PATH + '/' + SAMPLE_NAME + '.downstream.flank.removal.stats.txt')
    
    
    ## 8. Count number of times each barcode occurs ##
    
    # count number of identical barcodes in each sample
    
    # create temporary count file
    BARCODE_COUNT_TEMP = create_temp_file()
    
    run_command([BBMAP_PATH + 'kmercountexact.sh',
          'in=' + SAMPLE_PATH + '/' + SAMPLE_NAME + '.barcodes.fastq.gz',
          'out=' + BARCODE_COUNT_TEMP[1] + '.txt',
          'fastadump=f',
          'k=27',
          'rcomp=f'])
#took last argument out of this, which was 'ow=t'. I couldn't find this in the documentation
    
    ## 9. Create TSV file with barcode counts for each sequence ##
    
    # create three column barcode count file with sample name
    create_barcode_tsv(BARCODE_COUNT_TEMP[1] + '.txt',
                       SAMPLE_PATH + '/' + SAMPLE_NAME + '.Nef_RM9_barcode_counts.txt',
                       SAMPLE_NAME)
    
    # remove temporary files
    PLUS_STRAND_SAM[0].close()
    PLUS_STRAND_FASTQ[0].close()
    MINUS_STRAND_SAM[0].close()
    MINUS_STRAND_FASTQ[0].close()
    UNMAPPED_STRAND_SAM[0].close()
    UPSTREAM_FLANK_REMOVED[0].close()
    BARCODE_COUNT_TEMP
    
    # remove intermediate files
    # copies of these already exist in 02-renamed_fastq/
    #remove_file(SAMPLE_PATH + '/' + SAMPLE_NAME + '_R1.fastq.gz')
    #remove_file(SAMPLE_PATH + '/' + SAMPLE_NAME + '_R2.fastq.gz')
    #remove_file(SAMPLE_PATH + '/' + SAMPLE_NAME + '.merged.fastq.gz')
    #remove_file(SAMPLE_PATH + '/' + SAMPLE_NAME + '.containing_upstream_flank.fastq.gz')
    #remove_file(SAMPLE_PATH + '/' + SAMPLE_NAME + '.containing_barcode.fastq.gz')
    #remove_file(SAMPLE_PATH + '/' + SAMPLE_NAME + '.mapped.sam')
    #remove_file(SAMPLE_PATH + '/' + SAMPLE_NAME + '.oriented.fastq.gz')

--[2019-09-20 09:19:05] bin/bbmap/bbmerge.sh in=RM9_counts/848-03_S3_L001_R1_001.fastq.gz in2=RM9_counts/848-03_S3_L001_R2_001.fastq.gz out=RM9_counts/848-03_S3_L001.merged.fastq.gz qtrim=t ow=t--
--[2019-09-20 09:19:32] bin/bbmap/bbduk.sh in=RM9_counts/848-03_S3_L001.merged.fastq.gz outm=RM9_counts/848-03_S3_L001.containing_upstream_flank.fastq.gz literal=ACTTGGTAGGGGTATCAGTG k=20 hdist=1 ow=t--
--[2019-09-20 09:19:36] bin/bbmap/bbduk.sh in=RM9_counts/848-03_S3_L001.containing_upstream_flank.fastq.gz outm=RM9_counts/848-03_S3_L001.containing_barcode.fastq.gz literal=AGTTACAAATTGGCAATAGA k=20 hdist=1 ow=t--
--[2019-09-20 09:19:37] bin/bbmap/bbmap.sh in=RM9_counts/848-03_S3_L001.containing_barcode.fastq.gz outm=RM9_counts/848-03_S3_L001.mapped.sam ref=ref/M33262.fasta ow=t--
--[2019-09-20 09:19:54] bin/bbmap/splitsam.sh RM9_counts/848-03_S3_L001.mapped.sam /var/folders/lz/b8xzxdf952bck9bb57k2d4yr0000gq/T/tmpabekmgz0.sam /var/folders/lz/b8xzxdf952bck9bb57k2d4yr0000gq/T/tmp9bh11k1b.sam /v